In [31]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet

In [32]:
data = pd.read_csv('../data/macro_with_shift.csv')

In [33]:
tech = pd.read_csv('../data/technical_dataset.csv')

In [34]:
data.drop(columns=['Unnamed: 0', 'target_date'], inplace=True)

In [36]:
data

,date,target,DTB3,MCUMFN,CPIAUCSL,GEPU_current,IPG211111CN,Killian,IRLTLT01USM156N,M2SL,Close,CFNAI,UNRATE,Imports,Production,Stocks
0,1997-01-02,24.20,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,5.3,7218.20,6489.4,838890.8
1,1997-01-03,24.00,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,5.3,7218.20,6489.4,838890.8
2,1997-01-06,23.90,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,5.3,7218.20,6489.4,838890.8
3,1997-01-07,23.05,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,5.3,7218.20,6489.4,838890.8
4,1997-01-08,22.30,5.033810,82.0867,159.400,76.906476,98.3693,-4.346762,6.58,3834.6,786.16,0.26,5.3,7218.20,6489.4,838890.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6081,2021-03-25,63.03,0.031304,76.3791,265.028,212.409145,165.8403,24.093239,1.61,19853.7,3972.89,2.22,6.0,5686.25,10975.0,1138710.0
6082,2021-03-26,63.81,0.031304,76.3791,265.028,212.409145,165.8403,24.093239,1.61,19853.7,3972.89,2.22,6.0,5686.25,10975.0,1138710.0
6083,2021-03-29,65.00,0.031304,76.3791,265.028,212.409145,165.8403,24.093239,1.61,19853.7,3972.89,2.22,6.0,5686.25,10975.0,1138710.0
6084,2021-03-30,63.50,0.031304,76.3791,265.028,212.409145,165.8403,24.093239,1.61,19853.7,3972.89,2.22,6.0,5686.25,10975.0,1138710.0


In [37]:
tech

,date,MA1_9,MA2_9,MA3_9,MA1_12,MA2_12,MA3_12,MOM(1),MOM(2),MOM(3),MOM(6),MOM(9),MOM(12),OBV1_9,OBV2_9,OBV3_9,OBV1_12,OBV2_12,OBV3_12
0,1996-12-31,0,1,1,0,1,1,1,1,1,0,1,1,1,1,1,1,1,1
1,1997-01-02,1,0,1,1,0,1,0,1,1,0,0,1,1,1,1,1,1,1
2,1997-01-03,1,1,0,1,1,1,1,0,1,1,0,0,1,1,1,1,1,1
3,1997-01-06,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1
4,1997-01-07,0,0,0,0,0,0,1,1,1,1,0,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6081,2021-03-24,1,1,1,1,1,1,1,0,0,0,0,0,1,0,0,0,0,0
6082,2021-03-25,1,1,1,1,1,1,0,1,0,0,0,0,0,1,0,0,0,0
6083,2021-03-26,1,1,1,1,1,1,1,0,1,1,0,0,1,1,1,1,0,0
6084,2021-03-29,0,0,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1
